In [1]:
import os
os.chdir('C:/myroom/')
print(os.getcwd())

C:\myroom


In [3]:
#데이터 핸들링
import pandas as pd
import numpy as np

#전처리
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from statsmodels.stats.outliers_influence import variance_inflation_factor

#모델
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

#하이퍼 파라미터 튜닝: 보통 랜덤서치로 대략 파악한 다음 그리드 서치로 미세조정
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV


#평가
from sklearn.metrics import f1_score

In [7]:
import warnings
warnings.filterwarnings("ignore")

#pd.set_option('display.max_rows',None)
#pd.set_option('display.max_colwidth', None)

train= pd.read_csv("./nlp-getting-started/train.csv")
test= pd.read_csv("./nlp-getting-started/test.csv")
train.shape, test.shape
# ((7613, 5), (3263, 4))

train['target'].value_counts()
#1:실제 발생, 0: 발생하지 않음

target
0    4342
1    3271
Name: count, dtype: int64

In [8]:
test.isnull().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

In [12]:
# train data
print(train.isnull().sum(),'\n')
for i in range(len(train.columns)):
    print(train.columns[i],'의 결측치 비율:', round(train.isnull().sum().iloc[i]/len(train),3))

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64 

id 의 결측치 비율: 0.0
keyword 의 결측치 비율: 0.008
location 의 결측치 비율: 0.333
text 의 결측치 비율: 0.0
target 의 결측치 비율: 0.0


In [13]:
# test data
print(test.isnull().sum(),'\n')
for i in range(len(test.columns)):
    print(test.columns[i],'의 결측치 비율:', round(test.isnull().sum().iloc[i]/len(test),3))

id             0
keyword       26
location    1105
text           0
dtype: int64 

id 의 결측치 비율: 0.0
keyword 의 결측치 비율: 0.008
location 의 결측치 비율: 0.339
text 의 결측치 비율: 0.0


In [17]:
train=train.dropna(subset=['keyword'],axis=0) # 결측치 비율이 0.008로 큰 의미없다고 판단 =>제거

print(train.isnull().sum())


#아는 것
X_known=train[train['location'].notna()]

#모르는 것
X_unknown=train[train['location'].isnull()].drop('location',axis=1)

print('\n')
print('학습데이터의 location unique한 값의 종류:', X_known['location'].nunique()) #3341
print('학습 데이터의 크기',len(X_known))
len(train)

id             0
keyword        0
location    2472
text           0
target         0
dtype: int64


학습데이터의 location unique한 값의 종류: 3341
학습 데이터의 크기 5080


7552

In [26]:
# 키워드별 재난트윗일 확률 평균 계산

kw_mean1 = X_known.groupby('keyword')['target'].mean().sort_values()
risk_map1 = kw_mean1.to_dict()
X_known['keyword_score'] = X_known['keyword'].map(risk_map1)
kw_score1 =X_known['keyword_score']

text1 = X_known['text']

x_df = pd.DataFrame({'text':text1,'kw_score':kw_score1})

## 장소 20개만
top20= X_known['location'].value_counts().index[:20]
X_known['location']=X_known['location'].mask(~X_known['location'].isin(top20),'etc')


In [27]:
kw_mean2 = X_unknown.groupby('keyword')['target'].mean().sort_values()
risk_map2 = kw_mean2.to_dict()
X_unknown['keyword_score'] = X_unknown['keyword'].map(risk_map2)
kw_score2 =X_unknown['keyword_score']

text2 = X_unknown['text']

x_target_df = pd.DataFrame({'text':text2,'kw_score':kw_score2})

In [29]:
x_target_df

,text,kw_score
38,Barbados #Bridgetown JAMAICA ÛÒ Two cars set ...,0.142857
41,on the outside you're ablaze and alive\nbut yo...,0.142857
43,SOOOO PUMPED FOR ABLAZE ???? @southridgelife,0.142857
44,I wanted to set Chicago ablaze with my preachi...,0.142857
45,I gained 3 followers in the last week. You? Kn...,0.142857
...,...,...
7565,Wrecked tired but not gonna be asleep before 3??,0.058824
7568,The Riddler would be the best early-exit prima...,0.058824
7576,He just wrecked all of you http://t.co/y46isyZkC8,0.058824
7578,@jt_ruff23 @cameronhacker and I wrecked you both,0.058824


In [19]:
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader
import torch
import numpy as np

# 1. 트위터 특화 BERT 로드
model_name = "cardiffnlp/twitter-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 2. 문장 리스트를 CLS 임베딩으로 벡터화하는 함수
def get_cls_batch(texts, batch_size=16, max_len=128):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    all_embeddings = []
    dataloader = DataLoader(texts, batch_size=batch_size)

    for batch in dataloader:
        # 트윗 전처리
        batch = [text.replace("http://", "").replace("https://", "").strip() for text in batch]

        # 토크나이징
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=max_len).to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            cls_embeddings = outputs.last_hidden_state[:, 0, :] 
        all_embeddings.append(cls_embeddings.cpu().numpy())  

    return np.vstack(all_embeddings) 

In [42]:
x_df.shape

(5080, 769)

In [20]:
vector1 = get_cls_batch(x_df['text'].tolist())
vector2 = get_cls_batch(x_target_df['text'].tolist())

In [24]:
v1=StandardScaler().fit_transform(vector1)
v2=StandardScaler().fit_transform(pd.DataFrame(x_df['kw_score']))

v3=StandardScaler().fit_transform(vector2)
v4=StandardScaler().fit_transform(pd.DataFrame(x_target_df['kw_score']))

In [31]:
#데이터 병합
x_df = pd.DataFrame(np.concatenate([v1, v2], axis=1), index=X_known.index)
print('학습데이터의 크기:',x_df.shape)

x_target_df = pd.DataFrame(np.concatenate([v3, v4], axis=1), index=X_unknown.index)
print('타겟의 크기:',x_target_df.shape)

학습데이터의 크기: (5080, 769)
타겟의 크기: (2472, 769)


In [32]:
#학습데이터의 주성분 분석
p=pd.DataFrame(np.cumsum(PCA().fit(x_df).explained_variance_ratio_))
p[p.iloc[:,0]>=0.9]

,0
159,0.900682
160,0.901609
161,0.902523
162,0.903425
163,0.904321
...,...
764,0.999998
765,0.999999
766,0.999999
767,1.000000


In [33]:
#타겟의 주성분 분석
p=pd.DataFrame(np.cumsum(PCA().fit(x_target_df).explained_variance_ratio_))
p[p.iloc[:,0]>=0.9]

,0
151,0.900683
152,0.901670
153,0.902651
154,0.903612
155,0.904565
...,...
764,0.999998
765,0.999999
766,1.000000
767,1.000000


In [34]:
x_data = PCA(n_components=159).fit_transform(x_df)
x_data = pd.DataFrame(x_data,index=X_known.index)
x_target = PCA(n_components=159).fit_transform(x_target_df)
x_target = pd.DataFrame(x_target,index=X_unknown.index)

In [35]:
from sklearn.preprocessing import LabelEncoder
y=X_known['location'] 
lb=LabelEncoder()
y=lb.fit_transform(y)

In [43]:
x_train,x_test,y_train, y_test = train_test_split(x_df,y,test_size=0.3,random_state=123)
x_train,x_val,y_train, y_val = train_test_split(x_train,y_train,test_size=0.15,random_state=123)

In [49]:

DTCmodel=DecisionTreeClassifier(random_state=123).fit(x_train,y_train)
y_pred5=DTCmodel.predict(x_test)

train_score=DTCmodel.score(x_train,y_train)
validation_score=DTCmodel.score(x_val,y_val)

differ=round((train_score-validation_score),3)


if not 0<=differ<=0.05:
    print(differ,'=> 과적합')
else:
    print(differ,'=> 과적합 아님')


print('f1 score =',round(f1_score(y_test,y_pred5,average='weighted'),3)) 

0.198 => 과적합
f1 score = 0.816


In [50]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import f1_score

# 의사결정트리 모델을 위한 하이퍼파라미터 범위 설정
param_grid = {
    'criterion': ['gini', 'entropy'],  # 분할 기준
    'max_depth': [5, 10, 15, 20, None],  # 트리의 최대 깊이
    'min_samples_split': [2, 5, 10, 20],  # 분할을 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4, 10],  # 리프 노드에서 최소 샘플 수
    'max_features': [None, 'sqrt', 'log2'],  # 분할 시 고려할 최대 특성 수
}

# RandomizedSearchCV로 대략적인 최적 파라미터를 찾기
g1 = RandomizedSearchCV(DecisionTreeClassifier(random_state=123), param_distributions=param_grid,
                        n_iter=50, cv=5, random_state=42, n_jobs=-1)
g1.fit(x_train, y_train)

# GridSearchCV로 최적의 파라미터를 세밀하게 조정
g2 = GridSearchCV(DecisionTreeClassifier(random_state=123), param_grid=param_grid, cv=5, n_jobs=-1)
g2.fit(x_train, y_train)

# 최적의 파라미터 출력
print("RandomizedSearchCV best params:", g1.best_params_)
print("GridSearchCV best params:", g2.best_params_)

# 최적 파라미터로 모델 학습
best_params_randomized = g1.best_params_
best_params_grid = g2.best_params_

# RandomizedSearchCV 결과로 모델 학습
final_model_randomized = DecisionTreeClassifier(random_state=42, **best_params_randomized)
final_model_randomized.fit(x_train, y_train)

# GridSearchCV 결과로 모델 학습
final_model_grid = DecisionTreeClassifier(random_state=42, **best_params_grid)
final_model_grid.fit(x_train, y_train)

# 예측 수행
y_pred_randomized = final_model_randomized.predict(x_test)
y_pred_grid = final_model_grid.predict(x_test)

# F1-score 계산
f1_randomized = f1_score(y_test, y_pred_randomized, average='weighted')
f1_grid = f1_score(y_test, y_pred_grid, average='weighted')

# 결과 출력
print("RandomizedSearchCV 최적 파라미터로 학습한 모델 F1-score:", round(f1_randomized, 3))
print("GridSearchCV 최적 파라미터로 학습한 모델 F1-score:", round(f1_grid, 3))

# 훈련 데이터와 검증 데이터 점수 차이 계산 (과적합 확인)
train_score_randomized = final_model_randomized.score(x_train, y_train)
validation_score_randomized = final_model_randomized.score(x_val, y_val)

train_score_grid = final_model_grid.score(x_train, y_train)
validation_score_grid = final_model_grid.score(x_val, y_val)

# 점수 차이 계산
differ_randomized = round((train_score_randomized - validation_score_randomized), 3)
differ_grid = round((train_score_grid - validation_score_grid), 3)

# 과적합 여부 확인
if not 0 <= differ_randomized <= 0.05:
    print(f"RandomizedSearchCV 차이 ({differ_randomized}) => 과적합")
else:
    print(f"RandomizedSearchCV 차이 ({differ_randomized}) => 과적합 아님")

if not 0 <= differ_grid <= 0.05:
    print(f"GridSearchCV 차이 ({differ_grid}) => 과적합")
else:
    print(f"GridSearchCV 차이 ({differ_grid}) => 과적합 아님")

RandomizedSearchCV best params: {'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 5, 'criterion': 'gini'}
GridSearchCV best params: {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10}
RandomizedSearchCV 최적 파라미터로 학습한 모델 F1-score: 0.853
GridSearchCV 최적 파라미터로 학습한 모델 F1-score: 0.839
RandomizedSearchCV 차이 (0.001) => 과적합 아님
GridSearchCV 차이 (0.001) => 과적합 아님
